In [3]:
import pandas as pd 
import pysd
from pysd.translators.vensim.vensim_file import VensimFile
from typing import Dict, Tuple, List

In [ ]:
model_path = "CLD.mdl"
model = pysd.read_vensim(str(model_path), split_views=True, initialize=False)


c:\Users\tanryan\AppData\Local\anaconda3\Lib\site-packages\pysd\translators\vensim\vensim_file.py:199: UserWarning: Only a single view with no subviews was detected. The model will be built in a single file.
  warnings.warn(
c:\Users\tanryan\AppData\Local\anaconda3\Lib\site-packages\pysd\builders\python\python_expressions_builder.py:2285: SyntaxWarning: 'a' has no equation specified
  return builder.build(arguments)
c:\Users\tanryan\AppData\Local\anaconda3\Lib\site-packages\pysd\builders\python\python_expressions_builder.py:2285: SyntaxWarning: 'b' has no equation specified
  return builder.build(arguments)
c:\Users\tanryan\AppData\Local\anaconda3\Lib\site-packages\pysd\builders\python\python_expressions_builder.py:2285: SyntaxWarning: 'c' has no equation specified
  return builder.build(arguments)


In [ ]:
import re

def parse_mdl_simple_arrows(path):
    text = Path(path).read_text(errors="ignore")

    nodes = set()
    edges = []

    # Look for lines like: Arrow VarA,VarB
    pattern = re.compile(r"Arrow\s+([A-Za-z0-9 _\-]+),\s*([A-Za-z0-9 _\-]+)")
    for src, tgt in pattern.findall(text):
        nodes.add(src.strip())
        nodes.add(tgt.strip())
        edges.append((src.strip(), tgt.strip()))

    return sorted(nodes), edges

nodes, edges = parse_mdl_simple_arrows("CLD.mdl")
print("Nodes:", nodes[:10])
print("Edges:", edges[:10])


ValueError: No :VIEWS: section found in the mdl file

In [168]:
model_path = "ClimateModule_NaturalEmissionsX.mdl"
extract_equations_and_views(model_path)


views: Empty DataFrame
Columns: []
Index: []
raw:                          Variable  \
0    "Non-Linear Lifetime of N2O"   
1    "Non-Linear Lifetime of CH4"   
2    Net CH4 Emissions in MTonCH4   
3               Airborne Fraction   
4        Net Primary Productivity   
..                            ...   
339                  Watt per J s   
340                    FINAL TIME   
341                  INITIAL TIME   
342                       SAVEPER   
343                     TIME STEP   

                                              Equation  
0                          135 - RAMP(0.3, 1900, 1950)  
1    8 + RAMP( 0.01, 1900, 2000) - RAMP( 0.02, 2000...  
2    (Natural CH4 Emissions-Total CH4 Breakdown) / ...  
3    (Total C Emission + CH4 Oxidation + Flux Humus...  
4    CO2 to C*(Flux Atmosphere to Biomass-Flux Biom...  
..                                                 ...  
339                                                  1  
340                                              

c:\Users\tanryan\AppData\Local\anaconda3\Lib\site-packages\pysd\builders\python\python_model_builder.py:224: UserWarning: Variable 'Climate Action Year' is not declared as a workbench variable in any view. It will be added to the main module.
  warn(
c:\Users\tanryan\AppData\Local\anaconda3\Lib\site-packages\pysd\builders\python\python_model_builder.py:224: UserWarning: Variable 'Climate Policy Scenario' is not declared as a workbench variable in any view. It will be added to the main module.
  warn(
c:\Users\tanryan\AppData\Local\anaconda3\Lib\site-packages\pysd\builders\python\python_model_builder.py:224: UserWarning: Variable 'Current year' is not declared as a workbench variable in any view. It will be added to the main module.
  warn(
c:\Users\tanryan\AppData\Local\anaconda3\Lib\site-packages\pysd\builders\python\python_model_builder.py:224: UserWarning: Variable 'Total CH4 Emissions' is not declared as a workbench variable in any view. It will be added to the main module.
  warn(

In [167]:
from typing import Dict, List, Tuple
import pandas as pd
import pysd

def extract_equations_and_views(model_path) -> Tuple[pd.DataFrame, List[str]]:
    equations: Dict[str, str] = {}
    debug: List[str] = [f"Using PySD {getattr(pysd, '__version__', 'unknown')} for '{model_path}'"]

    # --- 1. Extract equations ---
    # First, try raw Vensim parse
    try:
        from pysd.translators.vensim.vensim_file import VensimFile  # type: ignore
        vf = VensimFile(str(model_path))
        vf.parse()
        sections = getattr(vf, "sections", []) or []
        debug.append(f"Raw parse sections: {len(sections)}")

        def _stringify_equation(obj):
            for attr in [
                "rhs", "equation", "expr", "expression", "raw_equation", "raw", "text", "value", "definition",
            ]:
                val = getattr(obj, attr, None)
                if val is not None:
                    return str(val)
            for method in ["to_vensim", "to_text", "to_string"]:
                fn = getattr(obj, method, None)
                if callable(fn):
                    try:
                        return str(fn())
                    except Exception:
                        pass
            try:
                return repr(obj)
            except Exception:
                return None

        for section in sections:
            elements = getattr(section, "elements", []) or []
            debug.append(f"Section elements: {len(elements)}")
            for element in elements:
                name = (
                    getattr(element, "name", None)
                    or getattr(element, "canon_name", None)
                    or getattr(getattr(element, "component", None) or object(), "name", None)
                )
                equation_text = _stringify_equation(element)
                component = getattr(element, "component", None)
                if (not equation_text or len(equation_text) < 2) and component is not None:
                    eq2 = _stringify_equation(component)
                    if eq2 and len(eq2) > 1:
                        equation_text = eq2
                if name and equation_text:
                    equations[str(name)] = str(equation_text).strip()
        debug.append(f"Raw parse equations extracted: {len(equations)}")
    except Exception as e:
        debug.append(f"Raw parse failed: {e}")

    # If raw parse failed, try translation-based model
    if not equations:
        try:
            model = pysd.read_vensim(str(model_path))
            debug.append("Translation succeeded")
        except Exception as e:
            debug.append(f"Translation failed: {e}")
            return pd.DataFrame(), debug

        # Try model.doc first
        doc = getattr(model, "doc", None)
        if doc is not None:
            try:
                name_col = "Real Name" if "Real Name" in doc.columns else None
                eq_col = "Equation" if "Equation" in doc.columns else None
                debug.append(f"model.doc rows: {len(doc)}; cols: {list(doc.columns)}")
                alt_eq_cols = [c for c in ["Eqn", "Formula", "Definition"] if c in getattr(doc, "columns", [])]
                if name_col and (eq_col or alt_eq_cols):
                    for _, row in doc.iterrows():
                        name = str(row[name_col]).strip()
                        eq_val = row[eq_col] if eq_col else row[alt_eq_cols[0]]
                        eq = str(eq_val).strip()
                        if name:
                            equations[name] = eq
            except Exception as e:
                debug.append(f"Reading model.doc failed: {e}")

        # If still empty, try namespace/components
        if not equations:
            try:
                possible_containers = [
                    getattr(model, "components", None),
                    getattr(model, "_namespace", None),
                    getattr(getattr(model, "components", None) or object(), "_namespace", None),
                    getattr(getattr(model, "components", None) or object(), "namespace", None),
                    getattr(getattr(model, "components", None) or object(), "elements", None),
                ]
                found_any = False
                for container in possible_containers:
                    if not container:
                        continue
                    if hasattr(container, "items"):
                        found_any = True
                        debug.append(f"Namespace container items: {len(list(container.items()))}")
                        for name, meta in container.items():
                            eq = None
                            if isinstance(meta, dict):
                                eq = meta.get("orig_eqn") or meta.get("equation") or meta.get("eqn")
                            if eq is None and hasattr(meta, "get"):
                                try:
                                    eq = meta.get("equation")
                                except Exception:
                                    pass
                            if eq:
                                equations[str(name)] = str(eq).strip()
                    else:
                        for attr in ["_namespace", "namespace", "elements"]:
                            ns = getattr(container, attr, None)
                            if hasattr(ns, "items"):
                                found_any = True
                                debug.append(f"Namespace via {attr}: {len(list(ns.items()))}")
                                for name, meta in ns.items():
                                    eq = None
                                    if isinstance(meta, dict):
                                        eq = meta.get("orig_eqn") or meta.get("equation") or meta.get("eqn")
                                    if eq:
                                        equations[str(name)] = str(eq).strip()
                if not found_any:
                    raise AttributeError("No namespace container found on model")
            except Exception as e:
                debug.append(f"Reading components namespace failed: {e}")

    # --- 2. Views from PySD ---
    try:
        models = pysd.read_vensim(str(model_path), split_views=True)
        all_elements = models.components._namespace  # Py name -> element

        view_mapping = []
        for view_name, py_vars in models.modules.items():  # py_vars are Python-safe names
            for py_var in py_vars:
                element = all_elements.get(py_var)
                if element is None:
                    real_var = py_var  # fallback
                else:
                    real_var = getattr(element, "canon_name", None) or getattr(element, "name", None)
                view_mapping.append({"Variable": real_var, "View": view_name})

        df_views = pd.DataFrame(view_mapping)
        debug.append(f"Views parsed: {len(df_views)} variables across {len(models.modules)} views")
    except Exception as e:
        debug.append(f"Parsing views failed: {e}")
        df_views = pd.DataFrame()

    # --- 3. Merge formulas with views ---
    df_raw = pd.DataFrame([{"Variable": k, "Equation": v} for k, v in equations.items()])
    if not df_views.empty:
        df_merged = pd.merge(df_raw, df_views, on="Variable", how="left")
        debug.append(f"Merged dataframe rows: {len(df_merged)}")
    else:
        df_merged = df_raw

    print("views:",df_views)
    print("raw:",df_raw)

    return #df_merged, debug


In [156]:
df_raw

,Variable,Section,Formula
0,"""Non-Linear Lifetime of N2O""",__main__,None
1,"""Non-Linear Lifetime of CH4""",__main__,None
2,Net CH4 Emissions in MTonCH4,__main__,None
3,Airborne Fraction,__main__,None
4,Net Primary Productivity,__main__,None
...,...,...,...
339,Watt per J s,__main__,None
340,FINAL TIME,__main__,None
341,INITIAL TIME,__main__,None
342,SAVEPER,__main__,None


In [ ]:
import re
from pathlib import Path
import tempfile
import pandas as pd
import pysd

# Path to your .mdl file
model_path = Path("FeliX3_YoGL_v25_fixedV1b.mdl")  # replace with your file

debug = []

# --- Views from PySD ---
df_views = pd.DataFrame(columns=["Variable", "View"])  # always define columns

# Clean the mdl text before parsing
text = model_path.read_text()
cleaned_text = re.sub(r"\(\d+,\d+\)\|", "", text)

tmp_cleaned_path = Path(tempfile.mkdtemp(prefix="vensim_cleaned_")) / model_path.name
tmp_cleaned_path.write_text(cleaned_text)

# Parse with PySD
models = pysd.read_vensim(str(tmp_cleaned_path), split_views=True)
py_to_vensim = {py_name: vensim_name for vensim_name, py_name in models.namespace.items()}

# Build view mapping
view_mapping = []
for view_name, py_vars in models.modules.items():
    for py_var in py_vars:
        vensim_name = py_to_vensim.get(py_var, py_var)
        view_mapping.append({"Variable": vensim_name, "View": view_name})

df_views = pd.DataFrame(view_mapping)
debug.append(f"Views parsed: {len(df_views)} variables across {len(models.modules)} views")

# Inspect results
print(debug)
df_views.head(10)


c:\Users\tanryan\AppData\Local\anaconda3\Lib\site-packages\pysd\builders\python\subscripts.py:350: UserWarning: 
Dimension given by subscripts:
	{'"15-19"', '"10-14"'}
is incomplete using SchoolEnrollment instead.
Subscript_dict:
	{'SupplyChain': ['Consumption', 'Distribution', 'Processing', 'PostHarvest', 'PrimaryProduction'], 'AllPlantButOther': ['Pulses', 'Grains', 'VegFruits'], 'AltProteins': ['AltPasMeat', 'AltCropMeat', 'AltDairy', 'AltEggs'], 'AltProteinTech': ['Plant', 'PrecFerm', 'Cult'], 'MapAltProteins': ['PasMeat', 'CropMeat', 'Dairy', 'Eggs'], 'PlantsPulsesVeg': ['Pulses', 'VegFruits'], 'MapAnimalFood': ['AltPasMeat', 'AltCropMeat', 'AltDairy', 'AltEggs'], 'YoGL cohorts': ['"20-24"', '"25-29"', '"30-34"', '"35-39"', '"40-44"', '"45-49"', '"50-54"', '"55-59"', '"60-64"', '"65-69"', '"70-74"', '"75-79"', '"80-84"', '"85-89"', '"90-94"', '"95-99"', '"100+"'], 'SchoolEnrollment': ['"5-9"', '"10-14"', '"15-19"', '"20-24"'], 'PrimaryEdButOldest': ['"10-14"', '"15-19"', '"20-24"'

In [244]:

#model_path = "ClimateModule_NaturalEmissions.mdl"
import pysd
from parsimonious.exceptions import IncompleteParseError

model_path = "FeliX3_YoGL_v25_fixedV1b.mdl"

from pathlib import Path
import re

path = Path(model_path)
text = path.read_text()
# Remove coordinate annotations
cleaned_text = re.sub(r"\(\d+,\d+\)\|", "", text)
cleaned_path = path.parent / f"{path.stem}_cleaned.mdl"
cleaned_path.write_text(cleaned_text)


model = pysd.read_vensim(str(cleaned_path), split_views=True, initialize=False)




#models = VensimFile(str(model_path))
#abs = models.get_abstract_model
#abs.__module__
#models.modules
#models.namespace
#models.components.Component
#models.view_elements()
#models.parse()
#models.parse_sketch(subview_sep=[])

c:\Users\tanryan\AppData\Local\anaconda3\Lib\site-packages\pysd\builders\python\python_model_builder.py:173: UserWarning: Variable 'Days in Year' is declared as a workbench variable in 'diet_change' but it has been already added in 'land_use'.
  warn(
c:\Users\tanryan\AppData\Local\anaconda3\Lib\site-packages\pysd\builders\python\python_model_builder.py:173: UserWarning: Variable 'Normal Carbon Emissions of Coal' is declared as a workbench variable in 'scenarios' but it has been already added in 'carbon_cycle'.
  warn(
c:\Users\tanryan\AppData\Local\anaconda3\Lib\site-packages\pysd\builders\python\python_model_builder.py:173: UserWarning: Variable 'Variable16 Technology' is declared as a workbench variable in 'scenarios' but it has been already added in 'economy_gdp'.
  warn(
c:\Users\tanryan\AppData\Local\anaconda3\Lib\site-packages\pysd\builders\python\python_model_builder.py:173: UserWarning: Variable 'Normal Carbon Emissions of Oil' is declared as a workbench variable in 'scenarios

In [245]:
model.modules

{'biodiversity': ['agriculture_biomass_production_land_ratio',
  'agriculture_biomass_production_on_biodiversity_elasticity',
  'biodiversity_impact_climate_damage_nonlinearity',
  'biodiversity_impact_climate_damage_scale',
  'biodiversity_impact_fertilizer_consumption_nonlinearity',
  'biodiversity_impact_fertilizer_consumption_scale',
  'biodiversity_impact_reference_fertilizer_consumption',
  'biodiversity_impact_reference_temperature',
  'extinction_factor',
  'forest_biomass_production_land_ratio',
  'forest_biomass_production_on_biodiversity_elasticity',
  'impact_of_agricultural_land_changes_on_biodiversity',
  'impact_of_agricultural_land_changes_on_biodiversity_elasticity',
  'impact_of_agriculture_biomass_production_on_biodiversity',
  'impact_of_biomass_production_on_biodiversity',
  'impact_of_climate_damage_on_biodiversity',
  'impact_of_fertilizer_consumption_on_biodiversity',
  'impact_of_forest_biomass_production_on_biodiversity',
  'impact_of_forest_land_changes_on_bi

In [114]:
sections = getattr(model, "sections", [])
print(f"Found {len(sections)} sections:")
for i, section in enumerate(sections):
    section_name = getattr(section, "name", "Unnamed Section")
    elements_count = len(getattr(section, "elements", []))
    print(f"  Section {i+1}: '{section_name}' with {elements_count} elements")

Found 1 sections:
  Section 1: '__main__' with 344 elements


In [98]:
 sections

In [ ]:
model.get_abstract_model()

AbstractModel(original_path=WindowsPath('ClimateModule_NaturalEmissions.mdl'), sections=(AbstractSection(name='__main__', path=WindowsPath('ClimateModule_NaturalEmissions.py'), type='main', params=[], returns=[], subscripts=[], elements=[AbstractElement(name='"Non-Linear Lifetime of N2O"', components=[AbstractComponent(subscripts=([], []), ast=ArithmeticStructure(operators=['-'], arguments=(135, CallStructure(function=ReferenceStructure(reference='ramp', subscripts=None), arguments=(0.3, 1900, 1950)))), type='Auxiliary', subtype='Normal')], units='Years', limits=(None, None), documentation=''), AbstractElement(name='"Non-Linear Lifetime of CH4"', components=[AbstractComponent(subscripts=([], []), ast=ArithmeticStructure(operators=['+', '-'], arguments=(8, CallStructure(function=ReferenceStructure(reference='ramp', subscripts=None), arguments=(0.01, 1900, 2000)), CallStructure(function=ReferenceStructure(reference='ramp', subscripts=None), arguments=(0.02, 2000, 2100)))), type='Auxiliar

In [93]:
import pandas as pd
from pysd.translators.vensim.vensim_file import VensimFile

def extract_variables_table(model_path):
    vf = VensimFile(str(model_path))
    vf.parse()
    vf.parse_sketch(subview_sep=[])   # keep flat views
    abstract_model = vf.get_abstract_model()

    rows = []

    def stringify_formula(obj):
        # Try to get something printable
        if hasattr(obj, "ast"):
            try:
                return obj.ast.dump()
            except Exception:
                return str(obj.ast)
        return "Unknown"

    for section in abstract_model.sections:
        for element in section.elements:
            comps = getattr(element, "components", None)

            if comps:  
                # Case 1: components is a dict
                if isinstance(comps, dict):
                    for name, comp in comps.items():
                        rows.append({
                            "View": getattr(comp, "view", "Unknown"),
                            "Variable": name,
                            "Formula": stringify_formula(comp)
                        })
                # Case 2: components is a list
                elif isinstance(comps, list):
                    for comp in comps:
                        rows.append({
                            "View": getattr(comp, "view", "Unknown"),
                            "Variable": getattr(comp, "name", None),
                            "Formula": stringify_formula(comp)
                        })
            else:
                # Element itself is a variable
                rows.append({
                    "View": getattr(element, "view", "Unknown"),
                    "Variable": getattr(element, "name", None),
                    "Formula": stringify_formula(element)
                })

    return pd.DataFrame(rows)


In [95]:
df = extract_variables_table("ClimateModule_NaturalEmissionsX.mdl")
df

,View,Variable,Formula
0,Unknown,None,"ArithmeticStructure:\n\t ['-'] (135, CallStruc..."
1,Unknown,None,"ArithmeticStructure:\n\t ['+', '-'] (8, CallSt..."
2,Unknown,None,ArithmeticStructure:\n\t ['/'] (ArithmeticStru...
3,Unknown,None,ArithmeticStructure:\n\t ['/'] (ArithmeticStru...
4,Unknown,None,"ArithmeticStructure:\n\t ['*', '*'] (Reference..."
...,...,...,...
339,Unknown,None,1
340,Unknown,None,2100
341,Unknown,None,1900
342,Unknown,None,1
